In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC

from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization

from Function import svm_function

In [2]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)


In [2]:
import time
import tqdm
import multiprocessing

from sklearn import metrics
from itertools import repeat

class multi_ensemble_svm():
    def __init__(self):
        self.model_array = None
        self.model_size = None
        self.hp = {
            "kernel": 'C_linear', "C":0, "logGamma":0, "degree":0, "coef0":0, "n":0.5, "max_iter":1e7
        }
    
    def train(self, data, label, ensemble_data_size=1, kernel='C_linear', C=0, logGamma=0, degree=0, coef0=0, n=0.5, max_iter=1e7):
        train_time = time.time()
        self.hp["kernel"] = kernel
        self.hp["C"] = C
        self.hp["logGamma"] = logGamma
        self.hp["degree"] = degree
        self.hp["coef0"] = coef0
        self.hp["n"] = n
        self.hp["max_iter"] = max_iter
        
        x, y = svm_function.ensemble_data(data, label, size=ensemble_data_size)
        self.model_size = len(x)
        
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        result = pool.starmap(self._svm_train, tqdm.tqdm(zip(x, y), total=self.model_size))
        self.model_array = result
        print("ensemble svm train: %.2fs" % (time.time() - train_time))
        pool.close()
        return None

    def _svm_train(self, d, l):
        arr = np.arange(len(l))
        np.random.shuffle(arr)
        d = d[arr]
        l = l[arr]
        
        m = svm_function.svm_train_model(d, l, self.hp["kernel"], self.hp["C"], self.hp["logGamma"], self.hp["degree"], self.hp["coef0"], self.hp["n"], max_iter=self.hp["max_iter"])
        return m
    
    def test(self, x, y):
        pred_y, pred_y_score = self.predict(x)
            
        return metrics.roc_auc_score(y, pred_y_score)
    
    def _svm_predict(self, m, x):
        return m.predict(x)
    
    def predict(self, x):
        output = None
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        output = pool.starmap(self._svm_predict, tqdm.tqdm(zip(self.model_array, repeat(x)), total=self.model_size))
        pred_y_score = np.sum(output, axis=0) / self.model_size
        pred_y = np.where(pred_y_score >= 0.5, 1, 0)
        
        pool.close()
        return pred_y, pred_y_score

def cv_mp_esvm(data_x, data_y, fold=5,
                kernel='C_rbf', 
                C=0, 
                logGamma=0, 
                degree=3, 
                coef0=0, 
                n=0, 
                size=10, 
                max_iter=1000):
    cv_x, cv_y = svm_function.CV_balanced(data_x, data_y, fold)
        
    auroc_array = []
    for i in range(fold):
        x_train, y_train, x_test, y_test = svm_function.cv_train_test(cv_x, cv_y, i)
        clf = multi_ensemble_svm()
        clf.train(x_train, y_train, ensemble_data_size=size, kernel=kernel, 
                C=C, 
                logGamma=logGamma, 
                degree=degree, 
                coef0=coef0, 
                n=n, 
                max_iter=max_iter)
        auroc_array.append(clf.test(x_test, y_test))
    return sum(auroc_array) / len(auroc_array)

In [10]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        
        auroc = cv_mp_esvm(self.X_train[:, selected], self.y_train, fold=5,
                    kernel='C_rbf', 
                    C=-0.24806890111201105, 
                    logGamma=-4.955918931113543, 
                    degree=3, 
                    coef0=0, 
                    n=0, 
                    size=1, 
                    max_iter=1000)
        
        # json_dict = svm_function.cv_esvm_perf(data_x=self.X_train[:, selected], data_y=self.y_train, fold=5,
        #                             kernel='C_rbf', 
        #                             C=-0.24806890111201105, 
        #                             logGamma=-4.955918931113543, 
        #                             degree=3, 
        #                             coef0=0, 
        #                             n=0, 
        #                             size=1, 
        #                             max_iter=1000)
        # auroc = json_dict['avg AUROC']
        # auroc = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=5, n_jobs=-1, scoring='roc_auc').mean()
        score = 1 - auroc
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)


In [20]:
X = np.load("data/merge_data/seq_rbp/transcript2pse_in_one*rbp_pvalue_cutoff10-3_-log10__train__x.npy")
y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")
feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one*rbp_pvalue__cutoff10-3.csv").columns.drop(["loc", "Transcript_stable_ID"])

In [5]:
X = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__x.npy")
y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")
feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one2rbp_pvalue_cutoff10-3_-log10.csv").columns.drop(["loc", "Transcript_stable_ID"])

In [ ]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
feature_names = dataset.feature_names

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

problem = SVMFeatureSelection(X_train, y_train)

task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)

selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(feature_names[selected_features].tolist()))

100%|██████████| 7/7 [00:00<00:00, 334.33it/s]

ensemble svm train: 0.70s



100%|██████████| 7/7 [00:00<00:00, 339.14it/s]

ensemble svm train: 0.69s



100%|██████████| 7/7 [00:00<00:00, 415.71it/s]

...

ensemble svm train: 1.15s



100%|██████████| 7/7 [00:00<00:00, 490.29it/s]

ensemble svm train: 1.13s



100%|██████████| 7/7 [00:00<00:00, 57.47it/s]

Number of selected features: 147
Selected features: 1, 2, 3, 4, 6, 11, 13, 14, 16, 18, 20, 22, 27, 28, 30, 31, 33, 36, 37, 39, 40, 41, 42, 46, 47, 48, 50, 52, 53, 56, 57, 58, 59, 60, 63, 66, 69, 70, 71, 72, 74, 79, 84, 85, 86, 88, 89, 91, 92, 93, 98, 100, 102, 107, 108, 110, 111, 113, 115, 119, 122, 124, 126, 129, 131, 132, 133, 134, 138, 141, 148, 149, 152, 154, 155, 156, 160, 165, 167, 169, 170, 171, 172, 174, 175, 176, 177, 178, 180, 182, 183, 184, 188, 191, 192, 195, M001_0.6, M004_0.6, M016_0.6, M019_0.6, M023_0.6, M026_0.6, M033_0.6, M035_0.6, M042_0.6, M044_0.6, M050_0.6, M051_0.6, M068_0.6, M070_0.6, M072_0.6, M073_0.6, M074_0.6, M081_0.6, M083_0.6, M086_0.6, M087_0.6, M089_0.6, M111_0.6, M142_0.6, M143_0.6, M144_0.6, M146_0.6, M147_0.6, M152_0.6, M154_0.6, M159_0.6, M162_0.6, M164_0.6, M167_0.6, M169_0.6, M176_0.6, M177_0.6, M178_0.6, M228_0.6, M262_0.6, M269_0.6, M273_0.6, M291_0.6, M317_0.6, M319_0.6, M345_0.6, M346_0.6, M348_0.6, M349_0.6, M352_0.6, M353_0.6


In [9]:
clf = multi_ensemble_svm()
clf.train(X_train[:, selected_features], y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    ensemble_data_size=10, 
                                    max_iter=1000)
roc_score = clf.test(X_test[:, selected_features], y_test)
print('Subset roc_score:', roc_score)

clf = multi_ensemble_svm()
clf.train(X_train, y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    ensemble_data_size=10, 
                                    max_iter=1000)
roc_score = clf.test(X_test, y_test)
print('All roc_score:', roc_score)

100%|██████████| 70/70 [00:00<00:00, 242.18it/s]


ensemble svm train: 1.02s


100%|██████████| 70/70 [00:09<00:00,  7.14it/s]


Subset roc_score: 0.631126630980261


100%|██████████| 70/70 [00:00<00:00, 168.94it/s]


ensemble svm train: 1.38s


100%|██████████| 70/70 [00:20<00:00,  3.36it/s]

All roc_score: 0.6598151555704249


In [50]:
model = svm_function.esvm_train_model(X_train[:, selected_features], y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    size=10, 
                                    max_iter=1000)
roc_score = model.test(X_test[:, selected_features], y_test)
print('Subset roc_score:', roc_score)

model = svm_function.esvm_train_model(X_train, y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    size=10, 
                                    max_iter=1000)
roc_score = model.test(X_test, y_test)
print('All roc_score:', roc_score)

auroc: 0.9980666270077335
ensemble svm step: 1/70 | 0.16s
auroc: 0.9996860334457003
ensemble svm step: 2/70 | 0.17s
auroc: 0.9969429572344504
ensemble svm step: 3/70 | 0.16s
auroc: 0.9998017053341265
ensemble svm step: 4/70 | 0.16s
auroc: 0.9994877387798269
ensemble svm step: 5/70 | 0.15s
auroc: 0.9978022341199022
ensemble svm step: 6/70 | 0.16s
auroc: 0.9990911494480799
ensemble svm step: 7/70 | 0.17s
auroc: 0.9985127900059488
ensemble svm step: 8/70 | 0.16s
auroc: 0.9986945601163328
ensemble svm step: 9/70 | 0.16s
auroc: 0.996496794236235
ensemble svm step: 10/70 | 0.15s
auroc: 0.9994877387798269
ensemble svm step: 11/70 | 0.15s
auroc: 0.9993059686694428
ensemble svm step: 12/70 | 0.16s
auroc: 0.9995538370017847
ensemble svm step: 13/70 | 0.16s
auroc: 0.9989093793376957
ensemble svm step: 14/70 | 0.16s
auroc: 0.9969594817899399
ensemble svm step: 15/70 | 0.16s
auroc: 0.9968933835679821
ensemble svm step: 16/70 | 0.16s
auroc: 0.9993720668914007
ensemble svm step: 17/70 | 0.17s
auroc: 

In [7]:
from sklearn import metrics

model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
decision_values = model_selected.decision_function(X_test[:, selected_features])
roc_score = metrics.roc_auc_score(y_test, decision_values)

print('Subset roc_score:', roc_score)

model_all.fit(X_train, y_train)
decision_values = model_all.decision_function(X_test)
roc_score = metrics.roc_auc_score(y_test, decision_values)
print('All Features roc_score:', roc_score)

Subset roc_score: 0.703412512546002
All Features roc_score: 0.7190950150552023


In [9]:
best_features, best_fitness

(array([0.        , 0.12563065, 0.        , 1.        , 0.38552953,
        0.        , 0.3020514 , 0.30713952, 0.        , 0.        ,
        0.06912101, 0.        , 1.        , 0.24903158, 0.45950409,
        1.        , 0.39143759, 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.42732089, 1.        , 0.        ,
        0.42873195, 1.        , 0.        , 0.07719092, 0.        ]),
 0.026315789473684178)